In [0]:
 dbutils.fs.mount(
     source='wasbs://demo1@datastorage123.blob.core.windows.net/',
     mount_point='/mnt/srini',
     extra_configs={
 'fs.azure.account.key.datastorage123.blob.core.windows.net': '6xEUxRWroFePxZjTYSsjxbm0OQXz4/T67SJ+36IQziH7RuvG1XY+JIK9QLskt7Y2Ovi4lRXp2BpY+AStiKJgPQ=='
    }
 )



Out[1]: True

In [0]:
dbutils.fs.unmount("/mnt/srini")

/mnt/srini has been unmounted.
Out[2]: True

In [0]:
from pyspark.sql import SparkSession


# Assuming 'sales_territory_df', 'sales_order_header_df', and 'sales_order_detail_df' are DataFrames containing the respective tables

# Sample data
sales_territory_df = spark.read.format("csv").option("header", "true").load("dbfs:/mnt/bronze-srini/Silver1/Sales/SalesTerritory/2024-03-01T17-43-19.730/Sales_SalesTerritory.csv/part-00000-tid-1734200700251997872-6dc4b393-eac5-4b2f-8462-0f1a7cbcee37-439-1-c000.csv")
sales_order_header_df = spark.read.format("csv").option("header", "true").load("dbfs:/mnt/bronze-srini/Silver1/Sales/SalesOrderHeader/2024-03-01T17-41-42.423/Sales_SalesOrderHeader.csv/part-00000-tid-9211036654975870261-09b37c9c-f26b-482d-9e4f-f4aa45399a2f-421-1-c000.csv")
sales_order_detail_df = spark.read.format("csv").option("header", "true").load("dbfs:/mnt/bronze-srini/Silver1/Sales/SalesOrderDetail/2024-03-01T17-41-16.179/Sales_SalesOrderDetail.csv/part-00000-tid-6382754599007715728-4135d760-d846-46d1-8fba-9d7081abbd9c-416-1-c000.csv")
# Join the DataFrames and perform the aggregation
result_df = sales_territory_df.join(sales_order_header_df,sales_territory_df["TerritoryID"] == sales_order_header_df["TerritoryID"],'inner') \
    .join(sales_order_detail_df,sales_order_header_df["SalesOrderID"] == sales_order_detail_df["SalesOrderID"]) \
    .groupBy(sales_order_header_df['TerritoryID'], sales_territory_df['name']) \
    .agg({"OrderQty": "sum", "LineTotal": "sum"}) \
    .withColumnRenamed("sum(OrderQty)", "No_of_orders_by_Region") \
    .withColumnRenamed("sum(LineTotal)", "Totalsales_Amt") \
    .orderBy(sales_order_header_df["TerritoryID"])

# Show the result
result_df.show(n=100)

# Stop the SparkSession


+-----------+--------------+----------------------+--------------------+
|TerritoryID|          name|No_of_orders_by_Region|      Totalsales_Amt|
+-----------+--------------+----------------------+--------------------+
|          1|     NORTHWEST|               36776.0|1.6084942547585826E7|
|         10|UNITED KINGDOM|               20099.0|   7670721.035475573|
|          2|     NORTHEAST|               19843.0|   6939374.481004976|
|          3|       CENTRAL|               19493.0|   7909009.005871946|
|          4|     SOUTHWEST|               59105.0|2.4184609600808784E7|
|          5|     SOUTHEAST|               18875.0|   7879655.072150952|
|          6|        CANADA|               49381.0|1.6355770454862706E7|
|          7|        FRANCE|               19906.0|    7251555.64692642|
|          8|       GERMANY|               13143.0|   4915407.595885147|
|          9|     AUSTRALIA|               18293.0|1.0655335959318217E7|
+-----------+--------------+----------------------+

In [0]:
from pyspark.sql import SparkSession


# Assuming 'sales_territory_df', 'sales_order_header_df', and 'sales_order_detail_df' are DataFrames containing the respective tables

# Sample data
sales_territory_df = spark.read.format("csv").option("header", "true").load("dbfs:/mnt/bronze-srini/Silver1/Sales/SalesTerritory/2024-03-01T17-43-19.730/Sales_SalesTerritory.csv/part-00000-tid-1734200700251997872-6dc4b393-eac5-4b2f-8462-0f1a7cbcee37-439-1-c000.csv")
sales_order_header_df = spark.read.format("csv").option("header", "true").load("dbfs:/mnt/bronze-srini/Silver1/Sales/SalesOrderHeader/2024-03-01T17-41-42.423/Sales_SalesOrderHeader.csv/part-00000-tid-9211036654975870261-09b37c9c-f26b-482d-9e4f-f4aa45399a2f-421-1-c000.csv")
sales_order_detail_df = spark.read.format("csv").option("header", "true").load("dbfs:/mnt/bronze-srini/Silver1/Sales/SalesOrderDetail/2024-03-01T17-41-16.179/Sales_SalesOrderDetail.csv/part-00000-tid-6382754599007715728-4135d760-d846-46d1-8fba-9d7081abbd9c-416-1-c000.csv")
special_offer_df = spark.read.format("csv").option("header", "true").load("dbfs:/mnt/bronze-srini/Silver1/Sales/SpecialOffer/2024-03-01T17-44-04.690/Sales_SpecialOffer.csv/part-00000-tid-4492394228989563617-1e8eb4a9-f383-42ef-bd17-aca9939302de-448-1-c000.csv")

# Join the DataFrames and perform the aggregation
result_df = sales_territory_df.join(sales_order_header_df,sales_territory_df["TerritoryID"] == sales_order_header_df["TerritoryID"],'inner') \
    .join(sales_order_detail_df,sales_order_header_df["SalesOrderID"] == sales_order_detail_df["SalesOrderID"]) \
    .join(special_offer_df,sales_order_detail_df["SpecialOfferID"] == special_offer_df["SpecialOfferID"]) \
    .groupBy(sales_order_header_df['TerritoryID'],sales_territory_df['name'],sales_order_detail_df['SpecialOfferID'], special_offer_df['DiscountPct']) \
    .agg({"OrderQty": "sum", "LineTotal": "sum"}) \
    .withColumnRenamed("sum(OrderQty)", "No_of_orders_by_Region") \
    .withColumnRenamed("sum(LineTotal)", "Totalsales_Amt") \
    .orderBy(sales_order_header_df['TerritoryID'])
# Show the result
result_df.show(n=100)

+-----------+--------------+--------------+-----------+----------------------+--------------------+
|TerritoryID|          name|SpecialOfferID|DiscountPct|No_of_orders_by_Region|      Totalsales_Amt|
+-----------+--------------+--------------+-----------+----------------------+--------------------+
|          1|     NORTHWEST|             9|        0.3|                  26.0|           4275.1254|
|          1|     NORTHWEST|             3|       0.05|                 954.0|        68697.437115|
|          1|     NORTHWEST|             4|        0.1|                 398.0|  10584.152999999998|
|          1|     NORTHWEST|             2|       0.02|                2355.0|   849794.4273000009|
|          1|     NORTHWEST|            11|       0.15|                  31.0|  414.89392499999997|
|          1|     NORTHWEST|            13|       0.15|                 257.0|   74350.06424999995|
|          1|     NORTHWEST|             7|       0.35|                  90.0|         49509.54125|


In [0]:
PD_PATH = "/mnt/bronze-srini/Silver1"

def list_files_recursively(directory):
    file_paths = []
   
    files = dbutils.fs.ls(directory)
    for file in files:
        if file.isDir():
            # If the item is a directory, recursively call the function
            file_paths.extend(list_files_recursively(file.path))
        else:
            # If the item is a file and has a .csv extension, append its path to the list
            if file.name.endswith(".csv"):
                file_paths.append(file.path)
    return file_paths
 
# Get list of all CSV files in the parent directory and its subdirectories
file_paths = list_files_recursively(PD_PATH)
print(file_paths)

['dbfs:/mnt/bronze-srini/Silver1/HumanResources/Department/2024-03-01T17-24-10.804/HumanResources_Department.csv/part-00000-tid-7733362801257520836-8aaa0f1e-8581-437c-b1c3-ab221d181cd0-2-1-c000.csv', 'dbfs:/mnt/bronze-srini/Silver1/HumanResources/Employee/2024-03-01T17-24-31.244/HumanResources_Employee.csv/part-00000-tid-1688383702003999188-de32aabd-603d-4a1d-b296-b29c73c4591e-5-1-c000.csv', 'dbfs:/mnt/bronze-srini/Silver1/HumanResources/EmployeeDepartmentHistory/2024-03-01T17-24-48.189/HumanResources_EmployeeDepartmentHistory.csv/part-00000-tid-5088626282336386504-fc75e2bf-e46a-4ca2-947f-46f2bc351ba3-8-1-c000.csv', 'dbfs:/mnt/bronze-srini/Silver1/HumanResources/EmployeePayHistory/2024-03-01T17-25-06.022/HumanResources_EmployeePayHistory.csv/part-00000-tid-6186437401925973646-58d7d3ce-15e7-406c-8229-22e9f7d420ed-11-1-c000.csv', 'dbfs:/mnt/bronze-srini/Silver1/HumanResources/JobCandidate/2024-03-01T17-25-22.047/HumanResources_JobCandidate.csv/part-00000-tid-5763947427835957589-5cf377bf-

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, year, month


for file in file_paths:

    # sales_df = spark.read.csv(file, header=True, inferSchema=True)
    # print(sales_df)
    fp1=  'dbfs:/mnt/bronze-srini/Silver1/Sales/SalesOrderHeader/2024-03-01T17-41-42.423/Sales_SalesOrderHeader.csv/part-00000-tid-9211036654975870261-09b37c9c-f26b-482d-9e4f-f4aa45399a2f-421-1-c000.csv'
   
        
    if file ==fp1 :
        sales_df = spark.read.csv(file, header=True, inferSchema=True)  
        print(sales_df)
        sales_df = sales_df.withColumn("TotalDue", sales_df["TotalDue"].cast("float"))
        sales_df = sales_df.withColumn("OrderDate", sales_df["OrderDate"].cast("date"))  
             # Group by CustomerID, year of OrderDate, and month of OrderDate, then calculate average purchase
        avg_purchase_df = sales_df.groupBy("CustomerID", year("OrderDate").alias("yearofpurchase"), month("OrderDate").alias("monthofpurchase")) \
             .agg(avg("TotalDue").alias("Avgpurchase")) \
             .orderBy("Avgpurchase", year("OrderDate").desc(), month("OrderDate").asc())  
            # Show the result
        avg_purchase_df.show()
        
        
      

DataFrame[SalesOrderID: int, RevisionNumber: int, OrderDate: timestamp, DueDate: timestamp, ShipDate: timestamp, Status: int, OnlineOrderFlag: boolean, SalesOrderNumber: string, PurchaseOrderNumber: string, AccountNumber: string, CustomerID: int, SalesPersonID: int, TerritoryID: int, BillToAddressID: int, ShipToAddressID: int, ShipMethodID: int, CreditCardID: int, CreditCardApprovalCode: string, CurrencyRateID: int, SubTotal: double, TaxAmt: double, Freight: double, TotalDue: double, Comment: string, rowguid: string, ModifiedDate: timestamp]
+----------+--------------+---------------+------------------+
|CustomerID|yearofpurchase|monthofpurchase|       Avgpurchase|
+----------+--------------+---------------+------------------+
|     30078|          2013|              6|1.5183000564575195|
|     18704|          2014|              1|2.5304999351501465|
|     11280|          2014|              1|2.5304999351501465|
|     12715|          2014|              1|2.5304999351501465|
|     28781

In [0]:
from pyspark.sql import SparkSession


# Assuming 'sales_territory_df', 'sales_order_header_df', and 'sales_order_detail_df' are DataFrames containing the respective tables

# Sample data
sales_territory_df = spark.read.format("csv").option("header", "true").load("dbfs:/mnt/bronze-srini/Silver1/Sales/SalesTerritory/2024-03-01T17-43-19.730/Sales_SalesTerritory.csv/part-00000-tid-1734200700251997872-6dc4b393-eac5-4b2f-8462-0f1a7cbcee37-439-1-c000.csv")
sales_order_header_df = spark.read.format("csv").option("header", "true").load("dbfs:/mnt/bronze-srini/Silver1/Sales/SalesOrderHeader/2024-03-01T17-41-42.423/Sales_SalesOrderHeader.csv/part-00000-tid-9211036654975870261-09b37c9c-f26b-482d-9e4f-f4aa45399a2f-421-1-c000.csv")
sales_order_detail_df = spark.read.format("csv").option("header", "true").load("dbfs:/mnt/bronze-srini/Silver1/Sales/SalesOrderDetail/2024-03-01T17-41-16.179/Sales_SalesOrderDetail.csv/part-00000-tid-6382754599007715728-4135d760-d846-46d1-8fba-9d7081abbd9c-416-1-c000.csv")
special_offer_df = spark.read.format("csv").option("header", "true").load("dbfs:/mnt/bronze-srini/Silver1/Sales/SpecialOffer/2024-03-01T17-44-04.690/Sales_SpecialOffer.csv/part-00000-tid-4492394228989563617-1e8eb4a9-f383-42ef-bd17-aca9939302de-448-1-c000.csv")

# Join the DataFrames and perform the aggregation
result_df = sales_territory_df.join(sales_order_header_df,sales_territory_df["TerritoryID"] == sales_order_header_df["TerritoryID"],'inner') \
    .join(sales_order_detail_df,sales_order_header_df["SalesOrderID"] == sales_order_detail_df["SalesOrderID"]) \
    .join(special_offer_df,sales_order_detail_df["SpecialOfferID"] == special_offer_df["SpecialOfferID"]) \
    .groupBy(sales_order_header_df['TerritoryID'],sales_territory_df['name'],sales_order_detail_df['SpecialOfferID'], special_offer_df['DiscountPct']) \
    .agg({"OrderQty": "sum", "LineTotal": "sum"}) \
    .withColumnRenamed("sum(OrderQty)", "No_of_orders_by_Region") \
    .withColumnRenamed("sum(LineTotal)", "Totalsales_Amt") \
    .orderBy(sales_order_header_df['TerritoryID'].desc())
# Show the result
result_df.show(n=100)
#df.orderBy(['Salary'], ascending = [True]).show() 

+-----------+--------------+--------------+-----------+----------------------+--------------------+
|TerritoryID|          name|SpecialOfferID|DiscountPct|No_of_orders_by_Region|      Totalsales_Amt|
+-----------+--------------+--------------+-----------+----------------------+--------------------+
|          9|     AUSTRALIA|             3|       0.05|                 149.0|          3392.49845|
|          9|     AUSTRALIA|            13|       0.15|                 159.0|   46064.67337499998|
|          9|     AUSTRALIA|             2|       0.02|                 781.0|    640155.423691998|
|          9|     AUSTRALIA|            14|        0.2|                 103.0|          78578.9472|
|          9|     AUSTRALIA|             1|        0.0|               17097.0|   9886873.221401192|
|          9|     AUSTRALIA|            16|        0.4|                   4.0|            271.1952|
|          8|       GERMANY|            13|       0.15|                 207.0|   60152.62049999995|


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum



# Read data from tables
sales_territory_df = spark.read.format("csv").option("header", "true").load("dbfs:/mnt/bronze-srini/Silver1/Sales/SalesTerritory/2024-03-01T17-43-19.730/Sales_SalesTerritory.csv/part-00000-tid-1734200700251997872-6dc4b393-eac5-4b2f-8462-0f1a7cbcee37-439-1-c000.csv")
sales_order_header_df = spark.read.format("csv").option("header", "true").load("dbfs:/mnt/bronze-srini/Silver1/Sales/SalesOrderHeader/2024-03-01T17-41-42.423/Sales_SalesOrderHeader.csv/part-00000-tid-9211036654975870261-09b37c9c-f26b-482d-9e4f-f4aa45399a2f-421-1-c000.csv")
sales_order_detail_df = spark.read.format("csv").option("header", "true").load("dbfs:/mnt/bronze-srini/Silver1/Sales/SalesOrderDetail/2024-03-01T17-41-16.179/Sales_SalesOrderDetail.csv/part-00000-tid-6382754599007715728-4135d760-d846-46d1-8fba-9d7081abbd9c-416-1-c000.csv")
special_offer_df = spark.read.format("csv").option("header", "true").load("dbfs:/mnt/bronze-srini/Silver1/Sales/SpecialOffer/2024-03-01T17-44-04.690/Sales_SpecialOffer.csv/part-00000-tid-4492394228989563617-1e8eb4a9-f383-42ef-bd17-aca9939302de-448-1-c000.csv")

# Convert 'TerritoryID' column to integer datatype
sales_territory_df = sales_territory_df.withColumn("TerritoryID", col("TerritoryID").cast("int"))

# Join the DataFrames and perform the aggregation
result_df = sales_territory_df.join(sales_order_header_df, "TerritoryID", 'inner') \
    .join(sales_order_detail_df, "SalesOrderID") \
    .join(special_offer_df, "SpecialOfferID") \
    .groupBy("TerritoryID", "name", "SpecialOfferID", "DiscountPct") \
    .agg(sum("OrderQty").alias("No_of_orders_by_Region"), sum("LineTotal").alias("Totalsales_Amt")) \
    .orderBy("TerritoryID")

# Show the result
result_df.show(n=100)


+-----------+--------------+--------------+-----------+----------------------+--------------------+
|TerritoryID|          name|SpecialOfferID|DiscountPct|No_of_orders_by_Region|      Totalsales_Amt|
+-----------+--------------+--------------+-----------+----------------------+--------------------+
|          1|     NORTHWEST|             8|        0.1|                  43.0|           651.01527|
|          1|     NORTHWEST|             4|        0.1|                 398.0|  10584.152999999998|
|          1|     NORTHWEST|            11|       0.15|                  31.0|  414.89392499999997|
|          1|     NORTHWEST|             5|       0.15|                  85.0|  1736.9898750000002|
|          1|     NORTHWEST|             3|       0.05|                 954.0|        68697.437115|
|          1|     NORTHWEST|             2|       0.02|                2355.0|   849794.4273000009|
|          1|     NORTHWEST|             9|        0.3|                  26.0|           4275.1254|


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum



# Read data from tables
sales_territory_df = spark.read.format("csv").option("header", "true").load("dbfs:/mnt/bronze-srini/Silver1/Sales/SalesTerritory/2024-03-01T17-43-19.730/Sales_SalesTerritory.csv/part-00000-tid-1734200700251997872-6dc4b393-eac5-4b2f-8462-0f1a7cbcee37-439-1-c000.csv")
sales_order_header_df = spark.read.format("csv").option("header", "true").load("dbfs:/mnt/bronze-srini/Silver1/Sales/SalesOrderHeader/2024-03-01T17-41-42.423/Sales_SalesOrderHeader.csv/part-00000-tid-9211036654975870261-09b37c9c-f26b-482d-9e4f-f4aa45399a2f-421-1-c000.csv")
sales_order_detail_df = spark.read.format("csv").option("header", "true").load("dbfs:/mnt/bronze-srini/Silver1/Sales/SalesOrderDetail/2024-03-01T17-41-16.179/Sales_SalesOrderDetail.csv/part-00000-tid-6382754599007715728-4135d760-d846-46d1-8fba-9d7081abbd9c-416-1-c000.csv")
special_offer_df = spark.read.format("csv").option("header", "true").load("dbfs:/mnt/bronze-srini/Silver1/Sales/SpecialOffer/2024-03-01T17-44-04.690/Sales_SpecialOffer.csv/part-00000-tid-4492394228989563617-1e8eb4a9-f383-42ef-bd17-aca9939302de-448-1-c000.csv")

# Convert 'TerritoryID' column to integer datatype
sales_territory_df = sales_territory_df.withColumn("TerritoryID", col("TerritoryID").cast("int"))

# Join the DataFrames and perform the aggregation
result_df = sales_territory_df.join(sales_order_header_df, "TerritoryID", 'inner') \
    .join(sales_order_detail_df, "SalesOrderID") \
    .join(special_offer_df, "SpecialOfferID") \
    .groupBy("TerritoryID", "name", "SpecialOfferID", "DiscountPct") \
    .agg(sum("OrderQty").alias("No_of_orders_by_Region"), sum("LineTotal").alias("Totalsales_Amt")) \
    .orderBy("TerritoryID")
# Write DataFrame to Azure SQL Database
#azure_sql_jdbc_url = "jdbc:sqlserver://AdvantureWorks.database.windows.net;database=AdvantureWorks"
#azure_sql_table = "dbo.Result_dataframe"
#azure_sql_user = "cognineazuresqlserver"
#azure_sql_password = "Cognine@9848"




jdbc_url = "jdbc:sqlserver://cognineazuresqlserver.database.windows.net:1433;database=AdvantureWorks"
 
# Define connection properties
connection_properties = {
    "user": "cognineazuresqlserver",
    "password": "Cognine@9848",
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

result_df.write.jdbc(url=jdbc_url, 
                      table='result_dataframe', 
                      mode="append", 
                      properties=connection_properties)



In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, year, month


for file in file_paths:

    # sales_df = spark.read.csv(file, header=True, inferSchema=True)
    # print(sales_df)
    fp1=  'dbfs:/mnt/bronze-srini/Silver1/Sales/SalesOrderHeader/2024-03-01T17-41-42.423/Sales_SalesOrderHeader.csv/part-00000-tid-9211036654975870261-09b37c9c-f26b-482d-9e4f-f4aa45399a2f-421-1-c000.csv'
   
        
    if file ==fp1 :
        sales_df = spark.read.csv(file, header=True, inferSchema=True)  
        print(sales_df)
        sales_df = sales_df.withColumn("TotalDue", sales_df["TotalDue"].cast("float"))
        sales_df = sales_df.withColumn("OrderDate", sales_df["OrderDate"].cast("date"))  
             # Group by CustomerID, year of OrderDate, and month of OrderDate, then calculate average purchase
        avg_purchase_df = sales_df.groupBy("CustomerID", year("OrderDate").alias("yearofpurchase"), month("OrderDate").alias("monthofpurchase")) \
             .agg(avg("TotalDue").alias("Avgpurchase")) \
             .orderBy("Avgpurchase", year("OrderDate").desc(), month("OrderDate").asc())  
            # Show the result
        avg_purchase_df.show()
        

jdbc_url = "jdbc:sqlserver://cognineazuresqlserver.database.windows.net:1433;database=AdvantureWorks"
 
# Define connection properties
connection_properties = {
    "user": "cognineazuresqlserver",
    "password": "Cognine@9848",
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

avg_purchase_df.write.jdbc(url=jdbc_url, 
                      table='Avg_purchase', 
                      mode="append", 
                      properties=connection_properties)
        
      

DataFrame[SalesOrderID: int, RevisionNumber: int, OrderDate: timestamp, DueDate: timestamp, ShipDate: timestamp, Status: int, OnlineOrderFlag: boolean, SalesOrderNumber: string, PurchaseOrderNumber: string, AccountNumber: string, CustomerID: int, SalesPersonID: int, TerritoryID: int, BillToAddressID: int, ShipToAddressID: int, ShipMethodID: int, CreditCardID: int, CreditCardApprovalCode: string, CurrencyRateID: int, SubTotal: double, TaxAmt: double, Freight: double, TotalDue: double, Comment: string, rowguid: string, ModifiedDate: timestamp]
+----------+--------------+---------------+------------------+
|CustomerID|yearofpurchase|monthofpurchase|       Avgpurchase|
+----------+--------------+---------------+------------------+
|     30078|          2013|              6|1.5183000564575195|
|     18704|          2014|              1|2.5304999351501465|
|     11280|          2014|              1|2.5304999351501465|
|     12715|          2014|              1|2.5304999351501465|
|     28781